In [1]:
import os
import sys
import collections
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
!pip install -U torchvision
!pip install pycocotools

     |████████████████████████████████| 4.0MB 4.6MB/s 
     |████████████████████████████████| 753.4MB 14kB/s 
ERROR: allennlp 0.9.0 requires flaky, which is not installed.
ERROR: allennlp 0.9.0 requires responses>=0.7, which is not installed.
ERROR: allennlp 0.9.0 has requirement spacy<2.2,>=2.1.0, but you'll have spacy 2.2.3 which is incompatible.
  Found existing installation: torch 1.3.0
    Uninstalling torch-1.3.0:
      Successfully uninstalled torch-1.3.0
  Found existing installation: torchvision 0.4.1a0+d94043a
    Uninstalling torchvision-0.4.1a0+d94043a:
      Successfully uninstalled torchvision-0.4.1a0+d94043a
     |████████████████████████████████| 1.5MB 4.9MB/s 
  Created wheel for pycocotools: filename=pycocotools-2.0.0-cp36-cp36m-linux_x86_64.whl size=286569 sha256=659128b081e5dcdd908a36c928ec8ae0a8d8b59e882729ac60f15a0e6a797a0f
  Stored in directory: /root/.cache/pip/wheels/dc/e6/36/0e1ae88c868eb42d3f92181b1c9bbd0b217a7ec3da6bd62e55
Successfully built pycocotools


In [3]:
!cp -r ../input/vision-references-detection/* ./

In [4]:
import torch
import torchvision
import transforms as T
import utils
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [5]:
def rle_decode(mask_rle, shape):
    shape = (shape[1], shape[0])
    s = mask_rle.split()
    # gets starts & lengths 1d arrays
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
    starts -= 1
    # gets ends 1d array
    ends = starts + lengths
    # creates blank mask image 1d array
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    # sets mark pixles
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    # reshape as a 2d mask image
    return img.reshape(shape).T  # Needed to align to RLE direction

In [6]:
def refine_masks(masks, labels):
    # Compute the areas of each mask
    areas = np.sum(masks.reshape(-1, masks.shape[-1]), axis=0)
    # Masks are ordered from smallest to largest
    mask_index = np.argsort(areas)
    # One reference mask is created to be incrementally populated
    union_mask = {k:np.zeros(masks.shape[:-1], dtype=bool) for k in np.unique(labels)}
    # Iterate from the smallest, so smallest ones are preserved
    for m in mask_index:
        label = labels[m]
        masks[:, :, m] = np.logical_and(masks[:, :, m], np.logical_not(union_mask[label]))
        union_mask[label] = np.logical_or(masks[:, :, m], union_mask[label])
    # Reorder masks
    refined = list()
    for m in range(masks.shape[-1]):
        mask = masks[:, :, m].ravel(order='F')
        rle = to_rle(mask)
        label = labels[m] - 1
        refined.append([masks[:, :, m], rle, label])
    return refined

In [7]:
# Convert data to run-length encoding
def to_rle(bits):
    rle = []
    pos = 0
    for bit, group in itertools.groupby(bits):
        group_list = list(group)
        if bit:
            rle.extend([pos, sum(group_list)])
        pos += len(group_list)
    return rle

Evaluation

In [8]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [9]:
test = pd.read_csv("../input/preprocessing/train_df_truncated.csv")
test = test[test['dataset']=='test'][['ImageId', 'EncodedPixels', 'Height', 'Width', 'Category']].reset_index().iloc[:int(len(test)*0.1)]

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
num_classes = 46 + 1
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [11]:
img_path = "../input/imaterialist-fashion-2019-FGVC6/train/"

In [12]:
model_ft = get_model_instance_segmentation(num_classes)
model_ft.to(device)
model_ft.load_state_dict(torch.load("../input/model-resnet-1epoch/model_resnet.bin"))

for param in model_ft.parameters():
    param.requires_grad = False
model_ft.eval()

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


MaskRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d()
          )
    

In [13]:
sub_list = []
missing_count = 0
submission = []
ctr = 0

tt = ToTensor()
for i, row in tqdm(test.iterrows(), total=len(test)):
    # loading image
    image_id = row['ImageId']
    file_path = os.path.join(img_path, image_id)
    img = Image.open(file_path).convert("RGB")
    img = img.resize((512, 512), resample=Image.BILINEAR)
    img = tt(img)
    
    
    result = model_ft([img.to(device)])[0]
    masks = np.zeros((512, 512, len(result["masks"])))
    for j, m in enumerate(result["masks"]):
        res = transforms.ToPILImage()(result["masks"][j].permute(1, 2, 0).cpu().numpy())
        res = np.asarray(res.resize((512, 512), resample=Image.BILINEAR))
        masks[:, :, j] = (res[:, :] * 255. > 127).astype(np.uint8)

    lbls = result['labels'].cpu().numpy()
    scores = result['scores'].cpu().numpy()

    best_idx = 0
    for scr in scores:
        if scr > 0.8:
            best_idx += 1

    if best_idx == 0:
        sub_list.append([test.loc[i, 'ImageId'], '1 1', 23])
        missing_count += 1
        continue

    if masks.shape[-1] > 0:
        masks = refine_masks(masks[:, :, :best_idx], lbls[:best_idx])
        for m, rle, label in masks:
            sub_list.append([test.loc[i, 'ImageId'], ' '.join(list(map(str, list(rle)))), label])
    else:
        sub_list.append([test.loc[i, 'ImageId'], '1 1', 23])
        missing_count += 1

100%|██████████| 28359/28359 [5:44:29<00:00,  1.37it/s]


In [14]:
submission_df = pd.DataFrame(sub_list)


In [15]:
submission_df.to_csv('final_submission.csv')

In [16]:
test.to_csv('test_data.csv')